In [ ]:
import papermill as pm
import scrapbook as sb
import pandas as pd
from datetime import datetime, timedelta
from ipywidgets import FloatProgress
import os

In [ ]:
connection_string = 'sqlite:///data/data.db'

In [ ]:
tokens = pd.read_sql(f'''
SELECT o.token, MAX(o.ts) as end_date
FROM ohlc o
GROUP BY o.token
''', connection_string)
tokens['end_date'] = pd.to_datetime(tokens['end_date'])
tokens['start_date'] = tokens['end_date'] - timedelta(days=14)
tokens

In [ ]:
base_output_folder = os.path.expanduser('outputs')

if not os.path.exists(base_output_folder):
        os.makedirs(base_output_folder)

In [ ]:
for token in tokens['token'].tolist():
    start = tokens[tokens['token'] == str(token)]['start_date'].values[0]
    end = tokens[tokens['token'] == str(token)]['end_date'].values[0]
    print(f'running for {token} for date {start} to {end}', end='\r')
    res = pm.execute_notebook(
        'Anirudh_Tunoori_template.ipynb',
        f'{base_output_folder}/market_analysis_{token}.ipynb',
        parameters = {
            'from_date': f'{start}',
            'to_date': f'{end}',
            'token': token,
            'connection_string': connection_string,
        }
    )

In [ ]:
nbs = sb.read_notebooks('outputs')

In [ ]:

summary_all_runs = pd.DataFrame([
    pd.Series({
        'token': scrap['token'].data,
        'from_date': scrap['from_date'].data,
        'to_date': scrap['to_date'].data,
        'avgHourlyReturn': scrap['prices'].data.hourlyReturn.mean(),
        'avgHourlyVolatility': scrap['prices'].data.hourlyVolatility.mean(),
        'maximumDrawdown': scrap['prices'].data.maxDrawdown.min(),
        'closePriceRange': scrap['prices'].data.close.iloc[-1] - scrap['prices'].data.close.iloc[0]}) 
    for book, scrap in nbs.notebook_scraps.items()
])
summary_all_runs